In [12]:
%%writefile aula11.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/count.h>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/replace.h>
#include <thrust/functional.h> // Necessário para thrust::minus e thrust::multiplies

int main() {
    // Abrindo o arquivo para leitura
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo!" << std::endl;
        return 1;
    }

    std::vector<float> stocks;
    float price;

    // Lê os dados do arquivo e armazena no vetor
    while (file >> price) {
        stocks.push_back(price);
    }

    file.close();

    // Converte o vetor padrão do C++ para um vetor da Thrust
    thrust::host_vector<float> h_stocks(stocks.begin(), stocks.end());
    thrust::device_vector<float> d_stocks = h_stocks;

    int n = d_stocks.size();

    // Verifica se há dados suficientes para calcular as variações
    if (n < 2) {
        std::cerr << "Dados insuficientes para calcular variações diárias!" << std::endl;
        return 1;
    }

    // Vetor para armazenar os ganhos diários
    thrust::device_vector<float> ganho_diario(n - 1);

    // Calcula as diferenças diárias usando thrust::transform
    thrust::transform(d_stocks.begin() + 1, d_stocks.end(), d_stocks.begin(), ganho_diario.begin(), thrust::minus<float>());

    // Contagem de quantos dias o preço subiu
    int num_increases = thrust::count_if(ganho_diario.begin(), ganho_diario.end(),
                                         [] __device__ (const float& x) { return x > 0; });

    std::cout << "Número de dias com aumento no preço: " << num_increases << std::endl;

    // Substitui valores negativos por zero
    thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), ganho_diario.begin(), thrust::placeholders::_1 < 0, 0.0f);

    // Soma os ganhos totais
    float total_increase = thrust::reduce(ganho_diario.begin(), ganho_diario.end(), 0.0f, thrust::plus<float>());

    // Calcula o aumento médio nos dias com aumento
    float avg_increase = (num_increases > 0) ? total_increase / num_increases : 0.0f;

    std::cout << "Aumento médio nos dias com alta: " << avg_increase << std::endl;

    // Calcula a média dos preços das ações
    float mean = thrust::reduce(d_stocks.begin(), d_stocks.end()) / n;

    // Vetor para armazenar as diferenças ao quadrado
    thrust::device_vector<float> squared_diff(n);

    // Usa um iterador constante para a média e calcula (x_i - μ)^2
    thrust::constant_iterator<float> mean_iter(mean);
    thrust::transform(d_stocks.begin(), d_stocks.end(), mean_iter, squared_diff.begin(),
                      thrust::minus<float>());
    thrust::transform(squared_diff.begin(), squared_diff.end(), squared_diff.begin(),
                      squared_diff.begin(), thrust::multiplies<float>());

    // Calcula a variância
    float variance = thrust::reduce(squared_diff.begin(), squared_diff.end()) / n;

    std::cout << "Variância: " << variance << std::endl;

    return 0;
}


Overwriting aula11.cu


In [15]:
!nvcc aula11.cu -o aula11 --extended-lambda

In [18]:
!./aula11

Número de dias com aumento no preço: 3056
Aumento médio nos dias com alta: 3.15251
Variância: 123792
